In [8]:
import pandas as pd
import numpy as np
import json

## Converting dataframe into json file from putting into MongoDB
---
<b>Tables Checklist: </b>
- retail price (retail_price_df)
- growers received (grower_received_grp)
- total production (production_df)
- domestic consumption (domestic_consumption_df)
- import (import_grp)
- re-export (reexport_grp)
- export (export_df)
- top quality coffee (top_coffee_df)
---
<b>Other things need to be checked</b>
1. country names in the table same as listed in map json file
2. grouping the countries in growers received (average all coffee types)


## Read worldmap json file and create a list for all the country name

In [9]:
worldmap_df = pd.read_csv("../Dataset/worldmap.csv", sep= ";")
worldmap_df

,Geo Point,Geo Shape,ISO 3 territory code,Status,ISO 3 country code,English Name,Continent of the territory,Region of the territory,ISO 3166-1 Alpha 2-Codes,French Name
0,"21.892740179494872, 33.74379108021756","{""coordinates"": [[[33.251040000000046, 21.9997...",NaN,Adm. by EGY,EGY,Ma'tan al-Sarra,Africa,Northern Africa,NaN,Ma'tan al-Sarra
1,"46.80256937605332, 8.234429172288662","{""coordinates"": [[[9.566720000000089, 47.54045...",CHE,Member State,CHE,Switzerland,Europe,Western Europe,CH,Suisse
2,"49.219115463825695, -2.1286489627456704","{""coordinates"": [[[-2.0149999999999295, 49.214...",NaN,UK Territory,GBR,Jersey,Europe,Northern Europe,NaN,Jersey
3,"47.592902606915196, 14.14019348879525","{""coordinates"": [[[16.946180000000084, 48.6190...",AUT,Member State,AUT,Austria,Europe,Western Europe,AT,Autriche
4,"39.69190982231605, -7.962187190752776","{""coordinates"": [[[-7.43184999999994, 37.25319...",PRT,Member State,PRT,Portugal,Europe,Southern Europe,PT,Portugal
...,...,...,...,...,...,...,...,...,...,...
251,"14.652550173716877, -61.02128794867484","{""coordinates"": [[[-60.816949999999906, 14.473...",MTQ,FR Territory,FRA,Martinique,Americas,Caribbean,MQ,Martinique
252,"3.7923669015472297, 109.7081944595275","{""coordinates"": [[[[111.41152000000005, 2.3763...",MYS,Member State,MYS,Malaysia,Asia,South-Eastern Asia,MY,Malaisie
253,"62.7898967841579, 16.739754033053558","{""coordinates"": [[[[16.43722000000008, 56.2113...",SWE,Member State,SWE,Sweden,Europe,Northern Europe,SE,Suède
254,"7.959848413760552, -1.2073013828375054","{""coordinates"": [[[1.1988900000000626, 6.10055...",GHA,Member State,GHA,Ghana,Africa,Western Africa,GH,Ghana


In [10]:
worldmap_df = worldmap_df.drop(columns= ["Geo Point", "ISO 3 territory code", "Status", "ISO 3 country code", "Continent of the territory",
                                           "Region of the territory", "ISO 3166-1 Alpha 2-Codes", "French Name"])
worldmap_df = worldmap_df.rename(columns= {"English Name" : "Country Name"})
worldmap_df

,Geo Shape,Country Name
0,"{""coordinates"": [[[33.251040000000046, 21.9997...",Ma'tan al-Sarra
1,"{""coordinates"": [[[9.566720000000089, 47.54045...",Switzerland
2,"{""coordinates"": [[[-2.0149999999999295, 49.214...",Jersey
3,"{""coordinates"": [[[16.946180000000084, 48.6190...",Austria
4,"{""coordinates"": [[[-7.43184999999994, 37.25319...",Portugal
...,...,...
251,"{""coordinates"": [[[-60.816949999999906, 14.473...",Martinique
252,"{""coordinates"": [[[[111.41152000000005, 2.3763...",Malaysia
253,"{""coordinates"": [[[[16.43722000000008, 56.2113...",Sweden
254,"{""coordinates"": [[[1.1988900000000626, 6.10055...",Ghana


### Retail Price

In [11]:
retail_price_df = pd.read_csv("../Dataset/international_coffee_organization/Price_data/Coffee_retail_price.csv")
retail_price_df.head()

,Country (US$/lb),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Austria,4.90,4.57,4.99,4.97,4.58,5.18,5.42,4.37,3.97,...,6.69,8.32,8.38,8.62,8.63,7.44,5.64,6.22,6.63,6.166070
1,Belgium,3.27,2.92,3.05,2.78,3.42,4.88,4.04,3.98,4.19,...,5.18,6.48,6.20,6.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.62,4.16,3.44,3.59,3.75,3.45,3.79,4.64,4.58,4.208544
3,Cyprus,2.83,2.80,2.87,2.60,3.18,4.26,4.13,4.04,4.61,...,5.21,6.06,6.36,6.47,6.48,5.30,5.30,5.50,5.79,5.393971
4,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.31,5.10,...,5.01,6.58,7.13,7.00,6.61,5.75,6.14,6.58,7.27,6.697826


In [15]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in retail_price_df["Country (US$/lb)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czech Republic', 'Denmark', 'Finland', 'France', 'Germany', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Japan', 'Norway', 'Russia', 'Switzerland', 'United Kingdom', 'United States of America']
 not match: []


In [ ]:
# checking all the format in each column
retail_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country (US$/lb)  28 non-null     object 
 1   1990              17 non-null     float64
 2   1991              17 non-null     float64
 3   1992              17 non-null     float64
 4   1993              18 non-null     float64
 5   1994              19 non-null     float64
 6   1995              21 non-null     float64
 7   1996              21 non-null     float64
 8   1997              24 non-null     float64
 9   1998              24 non-null     float64
 10  1999              26 non-null     float64
 11  2000              26 non-null     float64
 12  2001              26 non-null     float64
 13  2002              27 non-null     float64
 14  2003              27 non-null     float64
 15  2004              27 non-null     float64
 16  2005              27 non-null     float64
 17 

### Growers received

In [ ]:
grower_received_df = pd.read_csv("../Dataset/international_coffee_organization/Price_data/Prices_to _Growers.csv")
grower_received_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country (US cents/lb)  65 non-null     object 
 1   coffee types           65 non-null     object 
 2   1990                   51 non-null     float64
 3   1991                   49 non-null     float64
 4   1992                   52 non-null     float64
 5   1993                   51 non-null     float64
 6   1994                   51 non-null     float64
 7   1995                   52 non-null     float64
 8   1996                   49 non-null     float64
 9   1997                   49 non-null     float64
 10  1998                   49 non-null     float64
 11  1999                   45 non-null     float64
 12  2000                   46 non-null     float64
 13  2001                   44 non-null     float64
 14  2002                   45 non-null     float64
 15  2003    

In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in grower_received_df["Country (US cents/lb)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Colombia', 'Kenya', 'United Republic of Tanzania', 'Bolivia', 'Burundi', 'Cameroon', 'Costa Rica', 'Cuba', 'Democratic Republic of the Congo', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras', 'India', 'Jamaica', 'Madagascar', 'Malawi', 'Mexico', 'Nicaragua', 'Panama', 'Papua New Guinea', 'Peru', 'Rwanda', 'Sri Lanka', 'Uganda', 'Venezuela', 'Zambia', 'Zimbabwe', 'Angola', 'Brazil', 'Ethiopia', 'Indonesia', 'Philippines', 'Thailand', 'Vietnam', 'Angola', 'Benin', 'Brazil', 'Burundi', 'Cameroon', 'Central African Republic', 'Republic of the Congo', 'Ivory Coast', 'Democratic Republic of the Congo', 'Ecuador', 'Gabon', 'Ghana', 'Guinea', 'India', 'Indonesia', 'Liberia', 'Madagascar', 'Nigeria', 'Papua New Guinea', 'Philippines', 'Sierra Leone', 'Sri Lanka', 'United Republic of Tanzania', 'Thailand', 'Togo', 'Trinidad and Tobago', 'Uganda', 'Vietnam']
 not match: []


In [ ]:
# groupby countries -> average the amount of different coffee types
grower_received_grp = grower_received_df.groupby("Country (US cents/lb)").mean().reset_index()
grower_received_grp.head()


,Country (US cents/lb),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,85.6569,91.1153,44.9183,11.87080,NaN,NaN,NaN,NaN,NaN,...,44.38890,48.2936,52.26440,34.50765,50.3799,41.99435,89.39440,145.80605,110.47815,NaN
1,Benin,53.7609,22.8961,24.0067,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bolivia,51.7802,52.4230,43.5276,27.32830,79.9150,76.98670,57.17990,100.06890,131.0938,...,NaN,NaN,225.64450,201.97230,239.2465,234.79370,271.79440,471.61690,455.14600,858.2941
3,Brazil,44.5867,35.7439,36.3968,44.21055,99.9974,106.51135,83.06235,111.63265,91.4982,...,104.10645,164.7769,128.34625,93.71205,107.9303,89.00675,100.30415,104.34355,79.10075,68.9681
4,Burundi,58.0195,55.0206,53.1979,50.03630,51.0661,49.51050,46.21120,46.48590,43.7877,...,85.69280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Total Production

In [ ]:
production_df = pd.read_csv("../Dataset/international_coffee_organization/Supply/Total_production.csv")
production_df.columns = ["Country (In thousand 60kg bags)", "Species growing", "1990", "1991", "1992", "1993", "1994", "1995",
                         "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007",
                         "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
production_df.head()

,Country (In thousand 60kg bags),Species growing,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,R A,50.3450,79.3310,77.5200,32.6080,76.802,62.1090,70.925,64.330,...,34.9700,28.7150,32.7900,34.9350,39.4050,40.5150,44.8300,35.0060,41.9027,51.8419
1,Bolivia,A,122.7770,103.5360,120.2350,50.8230,116.944,142.4850,124.579,140.719,...,117.2249,131.8354,105.2812,119.9122,99.8766,84.2191,77.9835,83.8112,82.5687,81.2654
2,Brazil,A R,27285.6286,27293.4934,34603.3542,28166.9786,28192.047,18060.2022,29196.743,26148.004,...,55428.4102,48591.8289,55418.0012,54688.9664,53304.7669,52870.5876,56788.1784,52739.8179,65130.7997,58210.7127
3,Burundi,A R,487.3930,667.1990,620.2380,393.3540,664.143,433.9800,400.969,249.785,...,352.9776,204.1328,405.9615,163.2177,247.5500,269.4576,196.4128,202.1079,204.3391,272.4688
4,Ecuador,A R,1503.8150,2123.8240,1185.4800,2069.0070,2375.766,1888.2330,1992.914,1190.663,...,853.9798,825.4144,828.1024,665.5450,644.0112,644.4926,644.8845,623.5744,496.0001,558.9352


In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in production_df["Country (In thousand 60kg bags)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Angola', 'Bolivia', 'Brazil', 'Burundi', 'Ecuador', 'Indonesia', 'Madagascar', 'Malawi', 'Papua New Guinea', 'Paraguay', 'Peru', 'Rwanda', 'East Timor', 'Zimbabwe', 'Republic of the Congo', 'Cuba', 'Dominican Republic', 'Haiti', 'Philippines', 'United Republic of Tanzania', 'Zambia', 'Cameroon', 'Central African Republic', 'Colombia', 'Costa Rica', 'Ivory Coast', 'Democratic Republic of the Congo', 'El Salvador', 'Equatorial Guinea', 'Ethiopia', 'Gabon', 'Ghana', 'Guatemala', 'Guinea', 'Guyana', 'Honduras', 'India', 'Jamaica', 'Kenya', 'Laos', 'Liberia', 'Mexico', 'Nepal', 'Nicaragua', 'Nigeria', 'Panama', 'Sierra Leone', 'Sri Lanka', 'Thailand', 'Togo', 'Trinidad and Tobago', 'Uganda', 'Venezuela', 'Vietnam', 'Yemen']
 not match: []


In [ ]:
# checking all the format in each column
production_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country (In thousand 60kg bags)  55 non-null     object 
 1   Species growing                  55 non-null     object 
 2   1990                             55 non-null     float64
 3   1991                             55 non-null     float64
 4   1992                             55 non-null     float64
 5   1993                             55 non-null     float64
 6   1994                             55 non-null     float64
 7   1995                             55 non-null     float64
 8   1996                             55 non-null     float64
 9   1997                             55 non-null     float64
 10  1998                             55 non-null     float64
 11  1999                             55 non-null     float64
 12  2000                    

In [ ]:
# replacing A to arabica and R to robusta
production_df["Species growing"] = production_df["Species growing"].str.replace("A", "Arabica").str.replace("R", "Robusta")
production_df.head()

,Country (In thousand 60kg bags),Species growing,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,Robusta Arabica,50.3450,79.3310,77.5200,32.6080,76.802,62.1090,70.925,64.330,...,34.9700,28.7150,32.7900,34.9350,39.4050,40.5150,44.8300,35.0060,41.9027,51.8419
1,Bolivia,Arabica,122.7770,103.5360,120.2350,50.8230,116.944,142.4850,124.579,140.719,...,117.2249,131.8354,105.2812,119.9122,99.8766,84.2191,77.9835,83.8112,82.5687,81.2654
2,Brazil,Arabica Robusta,27285.6286,27293.4934,34603.3542,28166.9786,28192.047,18060.2022,29196.743,26148.004,...,55428.4102,48591.8289,55418.0012,54688.9664,53304.7669,52870.5876,56788.1784,52739.8179,65130.7997,58210.7127
3,Burundi,Arabica Robusta,487.3930,667.1990,620.2380,393.3540,664.143,433.9800,400.969,249.785,...,352.9776,204.1328,405.9615,163.2177,247.5500,269.4576,196.4128,202.1079,204.3391,272.4688
4,Ecuador,Arabica Robusta,1503.8150,2123.8240,1185.4800,2069.0070,2375.766,1888.2330,1992.914,1190.663,...,853.9798,825.4144,828.1024,665.5450,644.0112,644.4926,644.8845,623.5744,496.0001,558.9352


### Domestic Consumption

In [ ]:
domestic_consumption_df = pd.read_csv("../Dataset/international_coffee_organization/Supply/Domestic_consumption.csv")
domestic_consumption_df.columns = ["Country (In thousand 60kg bags)", "Species growing", "1990", "1991", "1992", "1993", "1994", "1995",
                                   "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007",
                                   "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
domestic_consumption_df.head()

,Country (In thousand 60kg bags),Species growing,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,R A,20.0,30.0,35.0,20.00,25.0,10.0,20.0,40.0,...,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
1,Bolivia,A,25.0,27.0,27.5,28.50,29.5,30.5,31.5,32.5,...,47.5,49.0,50.5,52.0,53.5,55.0,57.0,58.5,60.0,61.0
2,Brazil,A R,8200.0,8500.0,8900.0,9100.00,9300.0,10100.0,11000.0,11500.0,...,19132.0,19720.0,20330.0,20085.0,20333.0,20508.0,21225.0,21997.0,22200.0,22000.0
3,Burundi,A R,2.0,1.6,1.7,1.91,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,Ecuador,A R,350.0,350.0,350.0,350.00,350.0,350.0,300.0,300.0,...,150.0,150.0,150.0,155.0,155.0,155.0,155.0,155.0,155.0,149.0


In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in domestic_consumption_df["Country (In thousand 60kg bags)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Angola', 'Bolivia', 'Brazil', 'Burundi', 'Ecuador', 'Indonesia', 'Madagascar', 'Malawi', 'Papua New Guinea', 'Paraguay', 'Peru', 'Rwanda', 'East Timor', 'Zimbabwe', 'Republic of the Congo', 'Cuba', 'Dominican Republic', 'Haiti', 'Philippines', 'United Republic of Tanzania', 'Zambia', 'Cameroon', 'Central African Republic', 'Colombia', 'Costa Rica', 'Ivory Coast', 'Democratic Republic of the Congo', 'El Salvador', 'Equatorial Guinea', 'Ethiopia', 'Gabon', 'Ghana', 'Guatemala', 'Guinea', 'Guyana', 'Honduras', 'India', 'Jamaica', 'Kenya', 'Laos', 'Liberia', 'Mexico', 'Nepal', 'Nicaragua', 'Nigeria', 'Panama', 'Sierra Leone', 'Sri Lanka', 'Thailand', 'Togo', 'Trinidad and Tobago', 'Uganda', 'Venezuela', 'Vietnam', 'Yemen']
 not match: []


In [ ]:
# checking all the format in each column
domestic_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country (In thousand 60kg bags)  55 non-null     object 
 1   Species growing                  55 non-null     object 
 2   1990                             55 non-null     float64
 3   1991                             55 non-null     float64
 4   1992                             55 non-null     float64
 5   1993                             55 non-null     float64
 6   1994                             55 non-null     float64
 7   1995                             55 non-null     float64
 8   1996                             55 non-null     float64
 9   1997                             55 non-null     float64
 10  1998                             55 non-null     float64
 11  1999                             55 non-null     float64
 12  2000                    

In [ ]:
domestic_consumption_df["Species growing"] = domestic_consumption_df["Species growing"].str.replace("A", "Arabica").str.replace("R", "Robusta")
domestic_consumption_df.head()

,Country (In thousand 60kg bags),Species growing,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,Robusta Arabica,20.0,30.0,35.0,20.00,25.0,10.0,20.0,40.0,...,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
1,Bolivia,Arabica,25.0,27.0,27.5,28.50,29.5,30.5,31.5,32.5,...,47.5,49.0,50.5,52.0,53.5,55.0,57.0,58.5,60.0,61.0
2,Brazil,Arabica Robusta,8200.0,8500.0,8900.0,9100.00,9300.0,10100.0,11000.0,11500.0,...,19132.0,19720.0,20330.0,20085.0,20333.0,20508.0,21225.0,21997.0,22200.0,22000.0
3,Burundi,Arabica Robusta,2.0,1.6,1.7,1.91,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,Ecuador,Arabica Robusta,350.0,350.0,350.0,350.00,350.0,350.0,300.0,300.0,...,150.0,150.0,150.0,155.0,155.0,155.0,155.0,155.0,155.0,149.0


#### Import 

In [13]:
import_df = pd.read_csv("../Dataset/international_coffee_organization/Trade/all_import.csv")
import_df.head()

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Algeria,"1,058.887","1,782.264",862.266,"1,470.025","1,697.675",897.735,"1,229.068","1,461.023",475.395,...,"2,021.446","1,942.379","2,116.647","2,124.726","2,153.974","2,159.120","2,323.130","2,190.164","1,819.133","2,259.792"
1,Benin,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.780,...,8.087,12.718,7.674,6.309,10.640,15.769,11.973,15.570,22.396,12.339
2,Botswana,12.974,11.536,18.099,22.070,20.425,9.249,26.427,13.261,29.537,...,22.017,23.347,30.700,22.127,20.760,32.991,23.295,21.018,5.776,17.567
3,Burkina Faso,5.534,7.054,1.893,7.071,8.431,13.125,11.419,12.010,16.345,...,60.257,70.139,65.029,78.134,82.762,127.510,91.640,106.900,126.316,104.666
4,Cape Verde,1.670,7.119,17.500,9.968,8.026,12.591,13.053,7.706,8.742,...,11.342,10.558,12.326,11.682,11.226,11.107,12.259,13.529,13.943,11.236


In [14]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in import_df["Country (In thousand 60-kg bags)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

KeyError: 'English Name'

#### Some of the country is not in the map, so it was skipped but will still store into database

In [ ]:
# remove (,) in each cell
for column in import_df.columns:
    import_df[column] = import_df[column].str.replace(",","", regex = False)
 
import_df

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Algeria,1058.887,1782.264,862.266,1470.025,1697.675,897.735,1229.068,1461.023,475.395,...,2021.446,1942.379,2116.647,2124.726,2153.974,2159.120,2323.130,2190.164,1819.133,2259.792
1,Benin,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.780,...,8.087,12.718,7.674,6.309,10.640,15.769,11.973,15.570,22.396,12.339
2,Botswana,12.974,11.536,18.099,22.070,20.425,9.249,26.427,13.261,29.537,...,22.017,23.347,30.700,22.127,20.760,32.991,23.295,21.018,5.776,17.567
3,Burkina Faso,5.534,7.054,1.893,7.071,8.431,13.125,11.419,12.010,16.345,...,60.257,70.139,65.029,78.134,82.762,127.510,91.640,106.900,126.316,104.666
4,Cape Verde,1.670,7.119,17.500,9.968,8.026,12.591,13.053,7.706,8.742,...,11.342,10.558,12.326,11.682,11.226,11.107,12.259,13.529,13.943,11.236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Russia,0.000,0.000,1381.858,1794.946,1725.730,1728.144,1127.972,2185.288,1731.918,...,4155.257,4217.683,4174.745,4410.193,4746.950,4710.038,5232.881,5467.783,5287.542,5916.633
139,Switzerland,1170.508,1130.292,1101.251,1059.878,1099.406,1016.829,1111.627,970.772,1111.835,...,2317.606,2497.914,2477.957,2666.751,2643.106,2747.859,2816.026,2903.618,3086.019,3228.383
140,Tunisia,101.000,90.201,110.583,100.624,137.267,136.783,143.650,164.617,147.417,...,304.090,428.782,438.924,438.967,460.028,458.707,465.553,546.156,532.967,508.332
141,United Kingdom,2898.298,2805.914,3088.172,3196.981,3465.860,2806.723,2903.453,2929.338,3142.408,...,4301.908,4183.230,4126.054,4206.091,4318.696,4895.407,5052.293,4989.038,5687.860,5554.364


In [ ]:
# change cell type using astype
dic_columns_type = {}
for column in import_df.columns:
    if column != "Country (In thousand 60-kg bags)":
        dic_columns_type[column] = "float"
    
import_df = import_df.astype(dic_columns_type)
import_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country (In thousand 60-kg bags)  143 non-null    object 
 1   1990                              137 non-null    float64
 2   1991                              137 non-null    float64
 3   1992                              141 non-null    float64
 4   1993                              141 non-null    float64
 5   1994                              141 non-null    float64
 6   1995                              141 non-null    float64
 7   1996                              141 non-null    float64
 8   1997                              141 non-null    float64
 9   1998                              141 non-null    float64
 10  1999                              142 non-null    float64
 11  2000                              142 non-null    float64
 12  2001    

In [ ]:
# grouping the same country together
import_grp = import_df.groupby("Country (In thousand 60-kg bags)").mean().reset_index()
import_grp

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.250,0.250,0.250,0.250,0.250,0.250,0.867,2.900,1.000,...,9.665,1.040,4.637,3.033,7.792,11.539,10.681,34.357,38.556,8.158
1,Albania,28.417,9.283,33.333,59.685,67.802,31.302,11.275,1.651,6.248,...,113.653,116.812,123.346,131.379,123.046,143.464,154.180,154.907,125.143,144.686
2,Algeria,1058.887,1782.264,862.266,1470.025,1697.675,897.735,1229.068,1461.023,475.395,...,2021.446,1942.379,2116.647,2124.726,2153.974,2159.120,2323.130,2190.164,1819.133,2259.792
3,Anguilla,0.009,0.055,0.079,0.330,0.300,0.037,0.032,0.349,0.841,...,0.981,0.390,0.506,0.400,0.494,0.319,0.315,0.215,0.359,0.354
4,Antigua and Barbuda,0.718,0.602,0.303,0.261,0.321,0.332,0.107,1.103,0.170,...,1.950,3.443,3.459,2.165,2.090,2.833,1.862,4.492,4.445,5.282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,United States of America,21006.786,19839.630,22939.033,19328.721,16171.245,17106.857,19444.533,20342.746,21030.401,...,24378.013,26093.395,26056.163,27015.540,27565.498,27707.664,28837.838,29487.886,28918.207,30854.073
136,Uruguay,42.261,30.191,45.544,32.313,39.978,38.922,42.059,41.978,47.829,...,61.693,63.086,59.816,59.983,59.113,59.956,55.148,56.313,56.841,56.628
137,Uzbekistan,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,74.823,119.627
138,Vanuatu,0.411,0.269,0.578,0.352,0.454,0.318,0.417,0.530,0.211,...,1.568,1.558,0.941,1.778,1.580,1.483,1.615,3.637,6.124,2.180


#### Re-export

In [ ]:
reexport_df = pd.read_csv("../Dataset/international_coffee_organization/Trade/all_reexport.csv")
reexport_df.head()

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Austria,414.5930184,756.1850347,960.0820413,535.4880203,380.9950083,228.6660055,162.6780051,222.1238083,209.6159091,...,463.3459702,365.1472623,303.589908,310.8708473,386.3713576,377.5517359,307.6914777,308.2931411,257.4135488,274.1619791
1,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5053.064814,4893.973633,4753.595285,4256.978879,3952.812466,4459.903535,4550.04536,4377.324264,4377.326748,5013.520188
2,Luxembourg,890.8440337,1038.068039,1131.478041,1273.823045,1304.258046,1328.483041,1487.186551,1658.115555,2270.271439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bulgaria,166.4,183.459,7.3530002,70.4880001,109.9040006,122.0100008,25.685,19.2360001,17.9619999,...,133.0140903,121.9904582,183.8838149,190.8798212,244.4727217,275.4994882,326.8140544,291.8707366,311.5245159,361.8300649
4,Croatia,NaN,NaN,33.6450014,30.8840013,29.5880012,31.6510013,17.367,23.9010009,19.321001,...,23.9868757,23.7208153,24.1262407,26.2867961,51.2135581,62.5539035,99.0814808,77.9684384,85.5762211,85.9117545


In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in reexport_df["Country (In thousand 60-kg bags)"]:
    if name in worldmap_df["English Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Austria', 'Belgium', 'Luxembourg', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Japan', 'Norway', 'Russia', 'Switzerland', 'Tunisia', 'United Kingdom', 'United States of America', 'Algeria', 'Botswana', 'Burkina Faso', 'Cape Verde', 'Comoros', 'Djibouti', 'Egypt', 'Gambia', 'Guinea-Bissau', 'Lesotho', 'Mali', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Somalia', 'South Africa', 'Sudan', 'United Arab Emirates', 'Armenia', 'Australia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Brunei', 'Cambodia', 'China', 'China', 'China', 'United Arab Emirates', 'Fiji', 'Iran', 'Iraq', 'Israel', 'Jordan', 'Kazakhstan', 'Kuwait', 'Kyrgyzstan', 'Lebanon', 'Malaysia', 'Mongolia', 'Myanmar', 'New Zeal

In [ ]:
reexport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country (In thousand 60-kg bags)  130 non-null    object 
 1   1990                              117 non-null    object 
 2   1991                              119 non-null    object 
 3   1992                              126 non-null    object 
 4   1993                              127 non-null    object 
 5   1994                              127 non-null    object 
 6   1995                              127 non-null    object 
 7   1996                              127 non-null    object 
 8   1997                              127 non-null    object 
 9   1998                              127 non-null    object 
 10  1999                              129 non-null    object 
 11  2000                              129 non-null    object 
 12  2001    

In [ ]:
# remove (,) in each cell

for column in reexport_df.columns:
    if column != "2008":
        reexport_df[column] = reexport_df[column].str.replace(",","", regex = False)

reexport_df

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Austria,414.5930184,756.1850347,960.0820413,535.4880203,380.9950083,228.6660055,162.6780051,222.1238083,209.6159091,...,463.3459702,365.1472623,303.589908,310.8708473,386.3713576,377.5517359,307.6914777,308.2931411,257.4135488,274.1619791
1,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5053.064814,4893.973633,4753.595285,4256.978879,3952.812466,4459.903535,4550.04536,4377.324264,4377.326748,5013.520188
2,Luxembourg,890.8440337,1038.068039,1131.478041,1273.823045,1304.258046,1328.483041,1487.186551,1658.115555,2270.271439,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bulgaria,166.4,183.459,7.3530002,70.4880001,109.9040006,122.0100008,25.685,19.2360001,17.9619999,...,133.0140903,121.9904582,183.8838149,190.8798212,244.4727217,275.4994882,326.8140544,291.8707366,311.5245159,361.8300649
4,Croatia,NaN,NaN,33.6450014,30.8840013,29.5880012,31.6510013,17.367,23.9010009,19.321001,...,23.9868757,23.7208153,24.1262407,26.2867961,51.2135581,62.5539035,99.0814808,77.9684384,85.5762211,85.9117545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Canada,279.400,288.100,355.200,372.360,420.010,401.140,472.870,635.370,1093.380,...,779.710,967.890,945.130,1005.600,1078.890,1171.700,1250.660,1261.400,1287.940,1336.350
126,Argentina,14.810,3.020,1.110,2.940,1.250,68.000,125.230,128.600,2.880,...,5.190,5.300,4.250,4.030,3.820,4.240,4.340,5.230,1.130,3.400
127,Chile,4.560,6.150,11.860,40.500,61.190,62.900,112.840,83.940,71.670,...,65.500,52.580,17.810,46.830,53.100,55.620,30.340,22.530,19.810,18.940
128,Suriname,0.000,0.000,0.000,1.380,0.040,0.000,0.210,0.220,0.200,...,3.030,3.970,1.130,0.020,0.710,0.830,0.310,1.330,0.060,0.640


In [ ]:
# change cell type using astype
dic_columns_type = {}
for column in reexport_df.columns:
    if column != "Country (In thousand 60-kg bags)":
        dic_columns_type[column] = "float"
    
reexport_df = reexport_df.astype(dic_columns_type)
reexport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country (In thousand 60-kg bags)  130 non-null    object 
 1   1990                              117 non-null    float64
 2   1991                              119 non-null    float64
 3   1992                              126 non-null    float64
 4   1993                              127 non-null    float64
 5   1994                              127 non-null    float64
 6   1995                              127 non-null    float64
 7   1996                              127 non-null    float64
 8   1997                              127 non-null    float64
 9   1998                              127 non-null    float64
 10  1999                              129 non-null    float64
 11  2000                              129 non-null    float64
 12  2001    

In [ ]:
# grouping the same country together
reexport_grp = reexport_df.groupby("Country (In thousand 60-kg bags)").mean().reset_index()
reexport_grp

,Country (In thousand 60-kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Albania,0.000000,0.000000,0.000000,1.320000,1.980000,2.000000,2.000000,0.04000,2.99000,...,1.590000,0.590000,0.63000,3.680000,0.380000,0.370000,0.650000,1.780000,0.260000,2.410000
1,Algeria,19.350000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.040000,0.070000,0.08000,0.000000,0.010000,0.040000,0.250000,0.250000,0.480000,0.000000
2,Anguilla,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Antigua and Barbuda,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.01000,0.000000,0.000000,0.000000,0.010000,0.030000,0.210000,0.030000
4,Argentina,14.810000,3.020000,1.110000,2.940000,1.250000,68.000000,125.230000,128.60000,2.88000,...,5.190000,5.300000,4.25000,4.030000,3.820000,4.240000,4.340000,5.230000,1.130000,3.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,United States of America,825.168009,1010.450012,1376.580016,1719.584016,2572.141023,1840.986028,2048.532329,2446.29763,2446.60244,...,3190.871125,3728.167373,3383.74086,3247.904962,3360.741097,2958.179383,3174.963717,2929.672218,2902.389651,2842.417761
121,Uruguay,0.240000,0.210000,0.160000,0.010000,0.000000,0.000000,0.020000,0.01000,0.57000,...,0.030000,0.000000,0.84000,0.470000,0.010000,0.060000,0.000000,0.070000,0.790000,1.040000
122,Uzbekistan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
123,Vanuatu,0.610000,0.000000,0.860000,0.700000,0.380000,0.610000,0.670000,1.03000,1.86000,...,0.060000,0.680000,0.05000,0.040000,0.240000,0.240000,0.060000,0.000000,0.000000,0.000000


#### Export

In [ ]:
export_df = pd.read_csv("../Dataset/international_coffee_organization/Trade/Export.csv")
export_df.head()

,Country (In thousand 60kg bags),1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Angola,84.3500,70.5010,80.2500,38.878,8.3020,40.5590,51.831,50.49400,53.92900,...,4.370000,7.575000,8.375000,5.520000,9.375000,10.515000,10.945000,9.055000,9.323397,23.357178
1,Bolivia,156.4420,73.5230,96.2040,47.319,84.3210,93.9580,123.445,110.95500,97.03900,...,78.268006,74.308883,62.675780,54.850533,61.751267,30.280158,22.456342,26.119992,22.459634,20.341955
2,Brazil,16935.7876,21182.7614,18790.7192,17837.748,17273.1476,14468.4322,15250.609,16801.26001,18144.38833,...,33166.641590,33806.009330,28549.425890,31650.562950,37335.172830,37562.846750,34269.150250,30924.567850,35637.358590,40697.863710
3,Burundi,584.7730,687.8510,645.8580,417.609,507.8030,528.2020,224.076,528.76400,373.84100,...,307.118958,217.845799,392.006917,194.715883,252.178000,230.188550,204.270831,168.876264,201.725236,292.887291
4,Cameroon,2611.2590,1752.1790,1645.8510,704.530,545.8890,407.2690,563.549,1368.03000,745.71800,...,793.845667,490.283067,621.812800,271.949217,375.033867,390.142717,281.128967,245.017117,287.415250,249.902400


In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in export_df["Country (In thousand 60kg bags)"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['Angola', 'Bolivia', 'Brazil', 'Burundi', 'Cameroon', 'Central African Republic', 'Colombia', 'Republic of the Congo', 'Costa Rica', 'Ivory Coast', 'Cuba', 'Democratic Republic of the Congo', 'Dominican Republic', 'Ecuador', 'El Salvador', 'Equatorial Guinea', 'Ethiopia', 'Gabon', 'Ghana', 'Guatemala', 'Guinea', 'Guyana', 'Haiti', 'Honduras', 'India', 'Indonesia', 'Jamaica', 'Kenya', 'Laos', 'Liberia', 'Madagascar', 'Malawi', 'Mexico', 'Nepal', 'Nicaragua', 'Nigeria', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Rwanda', 'Sierra Leone', 'Sri Lanka', 'United Republic of Tanzania', 'Thailand', 'East Timor', 'Togo', 'Trinidad and Tobago', 'Uganda', 'Venezuela', 'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']
 not match: []


In [ ]:
export_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country (In thousand 60kg bags)  55 non-null     object 
 1   1990                             55 non-null     float64
 2   1991                             55 non-null     float64
 3   1992                             55 non-null     float64
 4   1993                             55 non-null     float64
 5   1994                             55 non-null     float64
 6   1995                             55 non-null     float64
 7   1996                             55 non-null     float64
 8   1997                             55 non-null     float64
 9   1998                             55 non-null     float64
 10  1999                             55 non-null     float64
 11  2000                             55 non-null     float64
 12  2001                    

In [ ]:
top_coffee_df = pd.read_csv("../Dataset/top_quality_coffee.csv")
top_coffee_df.head(2)

,coffee name,coffee rating,roaster name,roaster city,coffee origin country,coffee origin location,roast level,price,aroma,acidity,body,flavor,aftertaste,price USD per kg,roaster country
0,Colombia Pink Bourbon,95,modcup,Jersey City,Colombia,Piendamó;Cauca Department,Light,30.00,9,9,9,9,9,120.000000,United States of America
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taipei,Kenya,Kiambu County,Medium-Light,11.52,9,9,9,9,8,50.793651,Taiwan


In [ ]:
# checking country name in the table
# create two list to store all the country name that matching the json file or not matching the json file
match = []
not_match =[]

for name in top_coffee_df["roaster country"]:
    if name in worldmap_df["Country Name"].values:
        match.append(name)
    else:
        not_match.append(name)
print(f"match: {match}\n not match: {not_match}")

match: ['United States of America', 'Taiwan', 'Taiwan', 'Taiwan', 'United States of America', 'Mexico', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'United States of America', 'Taiwan', 'Taiwan', 'Taiwan', 'Taiwan', 'Taiwan', 'Taiwan', 'Taiwan', 'United States of America', 'United States of America', 'United States of America', 'Taiwan', 'United States of America', 'Taiwan', 'United States of America', 'United States of America', 'China', 'United States of America', 'United States of America', 'Taiwan', 'United States of America', 'United States of America', 'Taiwan', 'United States of America', 'United States of America', 'China', 'United States of America', 'China', 'United States of America', 'China', 'China', 'United States of America', 'United State

In [ ]:
# split origin country into list
top_coffee_df["coffee origin country"] = top_coffee_df["coffee origin country"].str.split(";")
top_coffee_df["coffee origin location"] = top_coffee_df["coffee origin location"].str.split(";")
top_coffee_df.head(2)

,coffee name,coffee rating,roaster name,roaster city,coffee origin country,coffee origin location,roast level,price,aroma,acidity,body,flavor,aftertaste,price USD per kg,roaster country
0,Colombia Pink Bourbon,95,modcup,Jersey City,[Colombia],"[Piendamó, Cauca Department]",Light,30.00,9,9,9,9,9,120.000000,United States of America
1,Kenya Kiambu Mandela Estate AA Washed Process,94,Buon Caffe,Taipei,[Kenya],[Kiambu County],Medium-Light,11.52,9,9,9,9,8,50.793651,Taiwan


### Export all the dataframe into json format

In [ ]:
# store data in json file
retail_price_df.to_json(path_or_buf = "./output_json/retail_price.json", orient= "records")
grower_received_grp.to_json(path_or_buf = "./output_json/grower_received.json", orient= "records")
production_df.to_json(path_or_buf = "./output_json/total_production.json", orient= "records")
domestic_consumption_df.to_json(path_or_buf = "./output_json/domestic_consumption.json", orient= "records")
import_grp.to_json(path_or_buf = "./output_json/import.json", orient= "records")
reexport_grp.to_json(path_or_buf = "./output_json/re-export.json", orient= "records")
export_df.to_json(path_or_buf = "./output_json/export.json", orient= "records")
top_coffee_df.to_json(path_or_buf = "./output_json/top_quality.json", orient= "records")
worldmap_df.to_json(path_or_buf = "./output_json/worldmap.json", orient= "records")
